In [6]:
import json
import numpy as np #
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


/kaggle/input/prelabeling-using-gpt/labeling_df_part1.csv
/kaggle/input/prelabeling-using-gpt/labeling_df_part5.csv
/kaggle/input/prelabeling-using-gpt/labeling_df_part3.csv
/kaggle/input/prelabeling-using-gpt/labeling_df_part2.csv
/kaggle/input/prelabeling-using-gpt/labeling_df_part4.csv


In [7]:
# Read the CSV file and select the first 5000 rows
unlabeled_df = pd.read_csv('/kaggle/input/prelabeling-using-gpt/labeling_df_part2.csv')
df = unlabeled_df.iloc[0:2000].copy()
#Save Token
df['content'] = df['content'].str.replace('<mentionstart>', '').str.replace('<mentionmiddle>', '').str.replace('<mentionend>', '')

In [8]:
df

,index,content
0,20000,"Ganjar-Mahfud paham banget, SMK Berasrama Grat..."
1,20001,Anies cocok jadi Presiden TikTok
2,20002,Mantap emang dukungan makin banyak untuk prabo...
3,20003,Ganjar-Mahfud jadi inspirasi buat pemimpin lai...
4,20004,"Iya sekali, DiTPSpastikan AYAHBowoGBRAN diang..."
...,...,...
1995,21995,"Prabowo malah promosi2in ganjarr, kocakk"
1996,21996,"Gue yakin banget, Ganjar Pranowo Mahfud MD bis..."
1997,21997,Sukses selalu untuk Ganjar-Mahfud yang mengins...
1998,21998,Bukti ketulusan hati Prabowo: dedikasinya yang...


In [9]:
def create_batch_file(df, batch_size, output_filename):
    system_text = """You will be provided with a tweet. Classify its sentiment, candidate mentioned, and topic using this metadata:

Sentiment Analysis:
    4 = None/Neutral/3 Candidate: Informational tweets outside election topic, spam, or ambiguous.
    3 = Comparative: Compares two candidates. Requires EXACTLY 2 entities.
        Example: "Prabowo is bad, Anies is good" 
    2 = Positive
    1 = Negative

Candidate Mention:
(Multiple Choice, sorted)
    0 = None: No candidate mentioned. 
    1 = Anies-Imin or other nickname/handle such si omon-omon, 01, etc.
    2 = Prabowo-Gibran or other nickname/handle such wowo, 02, etc.
    3 = Ganjar-Mahfud or other nickname/handle such paslon 03, etc.

Topic Discussed - Policy Issues:
    0 = None: Definition: Tweets that do not discuss any policy issues or candidate performance.
        Example: "Anies all in", "haha"

    1 = Economy, Industry, Infrastructure, Technology
    Definition: Tweets discussing economic policies, industry developments, infrastructure projects, or technological advancements.
    Criteria: Keywords related to economy, job creation, industry, infrastructure, or technology.

    2 = Education, Healthcare, Social Service
    Definition: Tweets discussing education policies, healthcare services, or social welfare programs."
    Criteria: Keywords related to education, healthcare, home issue, social help (bansos) and other wellfare.

    3 = Agriculture, Environment, Social Culture: 
    Definition: Tweets discussing agricultural policies, environmental issues, or social cultural topics.
    Criteria: Keywords related to agriculture, environment, or social culture.

    4 = Defense and Security, Foreign Policy
    Definition: Tweets discussing national defense, security measures, or foreign policy.
    Criteria: Keywords related to defense, security or foreign policy.

    5 = Corruption, Justice, Law: Corruption, justice system reforms, legal issues.
    Definition: Tweets discussing corruption, justice system reforms, or legal issues.
    Criteria: Keywords related to corruption, justice, or law including the law enforcement.

    6 = Candidate Personal Attributes, Ideology and Background
    Definition: Tweets discussing a candidate’s experience, educational background, or any scandals.
    Criteria: PERSONAL CANDIDATE context with keywords related to experience, education, or background and more. 

    7 = Supporter, Stakeholder, Parties:
    Definition: Tweets discussing supporters, stakeholders, or political parties.
    Criteria: existence of subject other than candidate with context and keywords related to supporters, stakeholders, or parties.

    8 = Recent Rallies/Speeches/Debate/Campaign Performance
    Definition: Tweets discussing recent rallies, speeches, debates, or overall campaign performance.
    Criteria: PERSONAL CANDIDATE context with keywords related to rallies, speeches, debates, or campaign events.

Format:
(tweet number).{"S":?,"C":[?],"T":?}
Example:
tweet1 = (comparative negative-positive, anies & ganjar, none)
tweet2 = (negative, anies, none)
Response:
1. {"S":4,"C":[1,3],"T":0}
2. {"S":3,"C":[1],"T":0}
"""
    with open(output_filename, 'w', encoding='utf-8') as outfile:
        for i in range(0, len(df), batch_size):
            batch_tweets = df['content'].iloc[i:i + batch_size].tolist()
            tweets_text = '\n'.join([f'Tweet {j+1}: {tweet}' for j, tweet in enumerate(batch_tweets)])
            request = {
                "custom_id": f"request-{i // batch_size + 1}",
                "method": "POST",
                "url": "/v1/chat/completions",
                "body": {
                    "model": "gpt-4o",
                    "messages": [
                        {"role": "system", "content": system_text},
                        {"role": "user", "content": tweets_text}
                    ],
                    "max_tokens": 4000,
                    "temperature": 0.3
                }
            }
            json.dump(request, outfile)
            outfile.write('\n')

# Usage
batch_size = 250
output_filename = 'batch_requests.jsonl'

create_batch_file(df, batch_size, output_filename)
# Karena API berbayar, hasil jsonl tersebut diupload "https://platform.openai.com/batches"